# How to apply tabular metrics and tests to text descriptors?

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

In [ ]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.test_suite import TestSuite

from evidently.metrics import ColumnSummaryMetric
from evidently.metrics import ColumnCorrelationsMetric
from evidently.metrics import ColumnDistributionMetric
from evidently.metrics import ColumnDriftMetric
from evidently.metrics import ColumnValueRangeMetric

from evidently.metrics import DatasetCorrelationsMetric
from evidently.metrics import DatasetDriftMetric
from evidently.metrics import DataDriftTable
from evidently.metrics import ClassificationQualityByFeatureTable
from evidently.metrics import RegressionErrorBiasTable

from evidently.tests import TestColumnDrift
from evidently.tests import TestValueRange
from evidently.tests import TestNumberOfOutRangeValues
from evidently.tests import TestShareOfOutRangeValues
from evidently.tests import TestMeanInNSigmas
from evidently.tests import TestColumnValueMin
from evidently.tests import TestColumnValueMax
from evidently.tests import TestColumnValueMean
from evidently.tests import TestColumnValueMedian
from evidently.tests import TestColumnValueStd
from evidently.tests import TestColumnQuantile 

from evidently.tests import TestHighlyCorrelatedColumns
from evidently.tests import TestTargetFeaturesCorrelations
from evidently.tests import TestPredictionFeaturesCorrelations
from evidently.tests import TestCorrelationChanges
from evidently.tests import TestNumberOfDriftedColumns
from evidently.tests import TestShareOfDriftedColumns

from evidently.descriptors import TextLength, TriggerWordsPresence, OOV, NonLetterCharacterPercentage

In [ ]:
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
#Data with Texts
reviews_data = datasets.fetch_openml(name='Womens-E-Commerce-Clothing-Reviews', version=2, as_frame='auto')
reviews = reviews_data.frame

In [ ]:
reviews['prediction'] = reviews['Rating']
reviews_ref = reviews[reviews.Rating > 3].sample(n=5000, replace=True, ignore_index=True, random_state=42) 
reviews_cur = reviews[reviews.Rating < 3].sample(n=5000, replace=True, ignore_index=True, random_state=42)

In [ ]:
column_mapping = ColumnMapping(
    numerical_features=['Age', 'Positive_Feedback_Count'],
    categorical_features=['Division_Name', 'Department_Name', 'Class_Name'],
    text_features=['Review_Text', 'Title']
)

## Colum Metrics with Descriptors

In [ ]:
table_column_metrics_report = Report(metrics=[
    ColumnDriftMetric(column_name = TextLength().for_column("Review_Text")),
    ColumnCorrelationsMetric(column_name = TextLength().for_column("Review_Text")),
    ColumnDistributionMetric(column_name = TextLength().for_column("Review_Text")),
    ColumnValueRangeMetric(column_name = TextLength().for_column("Review_Text"), left=0, right=20)
    
])

table_column_metrics_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
table_column_metrics_report

## Column Tests with Descriptors

In [ ]:
table_column_test_suite = TestSuite(tests=[
    TestColumnDrift(column_name = TextLength().for_column("Review_Text")),
    TestValueRange(column_name = TextLength().for_column("Review_Text")),
    TestNumberOfOutRangeValues(column_name = TextLength().for_column("Review_Text")),
    TestShareOfOutRangeValues(column_name = TextLength().for_column("Review_Text")),
    TestMeanInNSigmas(column_name = TextLength().for_column("Review_Text")),
    TestColumnValueMin(column_name = TextLength().for_column("Review_Text")),
    TestColumnValueMax(column_name = TextLength().for_column("Review_Text")),
    TestColumnValueMean(column_name = TextLength().for_column("Review_Text")),
    TestColumnValueMedian(column_name = TextLength().for_column("Review_Text")),
    TestColumnValueStd(column_name = TextLength().for_column("Review_Text")),
    TestColumnQuantile(column_name = TextLength().for_column("Review_Text"), quantile=0.25),
    
])

table_column_test_suite.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
table_column_test_suite

## Dataset Metrics with Descriptors
Currently, only default descriptors configuration is available

In [ ]:
table_column_metrics_report = Report(metrics=[
    DatasetDriftMetric(columns=["Age", "Review_Text"]),
    DataDriftTable(columns=["Age", "Review_Text"]), 
    
])

table_column_metrics_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
table_column_metrics_report

In [ ]:
regression_report = Report(metrics=[
    RegressionErrorBiasTable(columns=["Age", "Review_Text"]),
    
])

regression_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=ColumnMapping(
    target='Rating',
    prediction='prediction',
    numerical_features=['Age', 'Positive_Feedback_Count'],
    categorical_features=['Division_Name', 'Department_Name', 'Class_Name'],
    text_features=['Review_Text', 'Title'],
    task='regression'
  )
)
regression_report

In [ ]:
classification_report = Report(metrics=[
    ClassificationQualityByFeatureTable(columns=["Age", "Review_Text"]),

])
classification_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=ColumnMapping(
    target='Rating',
    prediction='prediction',
    numerical_features=['Age', 'Positive_Feedback_Count'],
    categorical_features=['Division_Name', 'Department_Name', 'Class_Name'],
    text_features=['Review_Text', 'Title'],
    task='classification'
  )
)

classification_report

## Dataset Tests with Descriptors
Currently, only default descriptors configuration is available

In [ ]:
table_dataset_test_suite = TestSuite(tests=[
    TestNumberOfDriftedColumns(columns=["Age", "Review_Text"]),
    TestShareOfDriftedColumns(columns=["Age", "Review_Text"]),
    
])

table_dataset_test_suite.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=ColumnMapping(
    target='Rating',
    prediction='prediction',
    numerical_features=['Age', 'Positive_Feedback_Count'],
    categorical_features=['Division_Name', 'Department_Name', 'Class_Name'],
    text_features=['Review_Text', 'Title'],
    task='regression'
  ))
table_dataset_test_suite